In [ ]:
import urllib.request
import json
import os
import ssl
import gradio as gr

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# 작성중
def iris(num1, num2, num3, num4):
    data = {
    "Inputs": {
        "input1": [
                {
                    "sepal_length": num1,
                    "sepal_width": num2,
                    "petal_length": num3,
                    "petal_width": num4,
                    "class": "unknown"
                }
            ]
    },
    "GlobalParameters": {}
    }

    body = str.encode(json.dumps(data))

    url = 'http://80205c5d-585c-4e07-9396-f3e8e99c5f4a.koreacentral.azurecontainer.io/score'
    # Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
    api_key = 'RTarqULijXzH0UajHCcgNSQQQaZBJCHF' # 원래는 숨겨야 한다. 하루만 쓰고 지울 것.
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")

    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

    req = urllib.request.Request(url, body, headers)
    try:
        response = urllib.request.urlopen(req)

        result_text = response.read().decode('utf-8')
        result_json = json.loads(result_text)
        assignments_value = result_json['Results']["WebServiceOutput0"][0]["Assignments"]

        return str(assignments_value)

    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        error_message = f"The request failed with status code: {error.code}\n{error.read().decode('utf8', 'ignore')}"
        return error_message

with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 붓꽃 종류 예측
        예측하고자 하는 붓꽃의 데이터를 입력하세요.
        """
        )
    with gr.Row():
        num1 = gr.Number(label="꽃받침의 길이")
        num2 = gr.Number(label="꽃받침의 폭")
    with gr.Row():
        num3 = gr.Number(label="꽃잎의 길이")
        num4 = gr.Number(label="꽃잎의 폭")
    
    submit_btn = gr.Button("제출")
    result = gr.Textbox(label="결과")

    submit_btn.click(iris, inputs=[num1, num2, num3, num4], outputs=[result])

demo.launch(share=True)
